In [11]:
import os
import json
import datetime
from dateutil import parser
from dataclasses import dataclass, field
import dataclasses
from TwitterAPI import TwitterAPI
from dacite import from_dict, Config
from typing import Optional

CREDENTIALS_FILE = "./creds.txt"
TWEETS_DIR_CONTROL = "../datasets/tweets/ia/nyc"
TWEETS_DIR = "../datasets/tweets/ia/hk"
OUTDIR = "../datasets/users/ia"
OUT_FILENAME = f"users.json"
CONTROL = False

if CONTROL:
    OUTDIR = f"{OUTDIR}/control"
    TWEETS_DIR = TWEETS_DIR_CONTROL

# TT_hongkong.nd json
# First Tweet retrieved Friday, August 9, 2019 8:33:46 PM EST
# Result_type: recent
OUTFILE = os.path.join(OUTDIR, OUT_FILENAME)

In [12]:
@dataclass
class Tweet:
    id: int
    text: str
    created_at: datetime.datetime
    lang: str
    source: str
    retweeted: bool

@dataclass
class User:
    id: int
    screen_name: str
    name: str
    description: Optional[str]
    location: str
    tweets: list[Tweet] = field(default_factory=list)

In [13]:
import glob
def all_lines(directory):
    for file in glob.glob(directory + "/*.json"):
        with open(file, "r") as f:
            for line in f:
                line = line.strip()
                if "REMOTE ARCHIVE NOT FOUND" in line:
                    break
                yield line
users = dict()
for line in all_lines(TWEETS_DIR):
    tweet = json.loads(line)
    user = tweet["user"]
    if "full_text" in tweet:
        tweet["text"] = tweet["full_text"]
    tweet["created_at"] = parser.parse(tweet["created_at"])
    if user["id"] not in users:
        users[user["id"]] = from_dict(data_class=User, data=user)
    users[user["id"]].tweets.append(
        from_dict(data_class=Tweet, data=tweet))


In [14]:
len(users)

35858

In [15]:
with open(OUTFILE, "w+") as f:
    for user in users.values():
        json.dump(dataclasses.asdict(user), f, default=str)
        f.write("\n")

In [16]:
print(len(users))
print(OUTFILE)

35858
../datasets/users/ia/users.json
